In [9]:
import math
import random

|n| Gausssian Elim   |      LU      |  np.linalg.det |
|-|-----------|:-------------:|------:|
|100| 37.3ms |  35.9ms | 1.26ms |
|500| 5.56s |  5.68s   |   27.6ms |

For copying:
```
tests = [([[1, -1, 3], [3, 3, -1], [-1, 1, 1]], 3, 6, 7, 3, 1),
 ([[3, 3, -1], [-1, 3, 1], [1, -1, 3]], 3, 6, 7, 3, 1),
 ([[3, -1], [-1, 5]], 2, 2, 7, 5, 3)]
```

# Questions
- What if matrix P is degenerate

# Remarks
- det works correctly, 30-200 times slower (for n=100, n=500)
- We don't need to mod all values in the resulting c
- for (100x100, 100) system the error is 1e-15

In [3]:
def determinant(matrix):
    """
    37.3ms vs 1.26ms (np.linalg.det) for 100x100
    Gaussian elimination O(n^3), almost the same as LU-decomposition by time
    Returns correct determinant
    might have an overflow, but it happens only for 1000x1000 matrices
    """
    # Get the size of the matrix
    n = len(matrix)
    
    # Create a copy of the matrix to avoid mutating the input
    mat = [row[:] for row in matrix]
    
    # Initialize determinant as 1
    det = 1
    
    for i in range(n):
        # Find pivot for column i
        pivot = i
        for j in range(i+1, n):
            if abs(mat[j][i]) > abs(mat[pivot][i]):
                pivot = j
        
        # If pivot is zero, determinant is zero
        if mat[pivot][i] == 0:
            return 0
        
        # Swap rows if needed
        if pivot != i:
            mat[i], mat[pivot] = mat[pivot], mat[i]
            det *= -1  # Swapping rows flips the sign of the determinant
        
        # Multiply determinant by the pivot element
        det *= mat[i][i]
        
        # Normalize row i
        for j in range(i+1, n):
            mat[i][j] /= mat[i][i]
        
        # Eliminate column i for rows below
        for j in range(i+1, n):
            for k in range(i+1, n):
                mat[j][k] -= mat[j][i] * mat[i][k]
    
    return det

In [4]:
def solve(A, b):
    """
    Solves the system of linear equations A * x = b using Gaussian elimination.
    
    Parameters:
    A (list of lists): Coefficient matrix (n x n).
    b (list): Right-hand side vector (n).
    
    Returns:
    list: Solution vector x (n).
    """
    n = len(A)
    
    # Forward elimination: Reduce to upper triangular form
    for i in range(n):
        # Find the pivot
        pivot = i
        for j in range(i + 1, n):
            if abs(A[j][i]) > abs(A[pivot][i]):
                pivot = j
        
        # Swap rows in A and b
        if pivot != i:
            A[i], A[pivot] = A[pivot], A[i]
            b[i], b[pivot] = b[pivot], b[i]
        
        # Check for singular matrix
        if A[i][i] == 0:
            raise ValueError("Matrix is singular or nearly singular.")
        
        # Eliminate entries below the pivot
        for j in range(i + 1, n):
            factor = A[j][i] / A[i][i]
            for k in range(i, n):
                A[j][k] -= factor * A[i][k]
            b[j] -= factor * b[i]
    
    # Back substitution: Solve for x in Ux = c
    x = [0] * n
    for i in range(n - 1, -1, -1):
        x[i] = b[i]
        for j in range(i + 1, n):
            x[i] -= A[i][j] * x[j]
        x[i] /= A[i][i]
    
    return x

In [5]:
def isPrime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True
    
# Use eratosthenes sieve
def next_prime(n):
    while True:
        if isPrime(n):
            return n
        n += 1

def hdet(gamma, alpha, weights):  
    H_matrix = [[alpha[i][j] * (gamma ** weights[i][j]) * (weights[i][j] != -1) for j in range(n)] for i in range(n)]
    return determinant(H_matrix) % prime    

def get_P(r_vals, gammas, prime):
    return [[pow(g, i, prime) for i in range(len(gammas))] for g in gammas]

In [6]:
def solver(weights, n, m, b, t, c):
    max_wt = max(t, c)
    min_siz = max(max_wt * n + 1, n * n)
    prime = next_prime(min_siz + 1)
    
    set_of_vals = list(range(prime))
    
    num_runs = math.ceil(math.log(n))
    for _ in range(num_runs):
        alpha = [[random.choice(set_of_vals[1:]) for _ in range(n)] for _ in range(n)]
        gammas = random.sample(set_of_vals[1:], n * max_wt + 1)
    
        r_vals =  [
            hdet(gamma, alpha, weights) % prime
            for gamma in gammas
        ]
    
        P_matrix = get_P(r_vals, gammas, prime)
        
        c = solve(P_matrix, r_vals)
    
        if c[b] % prime != 0:
            print("yes")
            return 0
    
    print("no")
    return 0

In [7]:
def main():
    n, m, b, t, c = map(int, input().split())
    weights = [[-1] * n for _ in range(n)]  
    for _ in range(m):
        u, v, w = map(int, input().split())
        weights[u][v] = w
    return solver(weights, n, m, b, t, c)

In [71]:
for t in tests:
    solver(*t)

yes
yes


IndexError: list index out of range

---

In [11]:
import numpy as np

In [2]:
tests = [([[1, -1, 3], [3, 3, -1], [-1, 1, 1]], 3, 6, 7, 3, 1),
 ([[3, 3, -1], [-1, 3, 1], [1, -1, 3]], 3, 6, 7, 3, 1),
 ([[3, -1], [-1, 5]], 2, 2, 7, 5, 3)]

In [28]:
np.random.randint(0,3, 10)

array([0, 0, 2, 0, 1, 2, 0, 0, 1, 0])

In [29]:
matrices = np.random.randn(10, 20,20)

In [35]:
prime, n, max_wt = 11, 3,3

In [36]:
min_siz = max(max_wt * n + 1, n * n)

In [37]:
set_of_vals = np.arange(prime)

In [38]:
alpha = np.random.randint(1,prime+1, (n,n))

In [52]:
gammas = np.random.choice(set_of_vals[1:], size=n * max_wt + 1, replace=False)

In [58]:
weights = np.array([[3, 3, -1], [-1, 3, 1], [1, -1, 3]])

In [86]:
r_vals =  np.linalg.det((gammas[:, None, None].astype(float))**weights[None, ...]*(weights != -1))

In [85]:
P_matrix = gammas[:,None] ** np.arange(len(gammas))[None,:]

In [123]:
r_vals =  np.linalg.det((gammas[:, None, None].astype(float))**weights[None, ...]*(weights != -1)).astype(int)

In [122]:
r_vals.dtype

dtype('float64')

In [94]:
def npsolver(weights, n, m, b, t, c):
    max_wt = max(t, c)
    min_siz = max(max_wt * n + 1, n * n)
    prime = next_prime(min_siz + 1)
    
    set_of_vals = np.arange(prime)

    weights = np.array(weights)
    
    num_runs = math.ceil(math.log(n))
    for _ in range(num_runs):
        alpha = np.random.randint(1,prime+1, (n,n))
        gammas = np.random.choice(set_of_vals[1:], size=n * max_wt + 1, replace=False)
        r_vals =  np.linalg.det((gammas[:, None, None].astype(float))**weights[None, ...]*(weights != -1)).astype(int)
    
        P_matrix = gammas[:,None] ** np.arange(len(gammas))[None,:]
        
        c = np.linalg.solve(P_matrix, r_vals)
    
        if c[b] % prime != 0:
            print("yes")
            return 0
    
    print("no")
    return 0

In [1]:
t, c = 3, 5
n = 10

In [154]:
npsolver(*tests[2])

yes


0

In [1]:
max_wt = max(t, c)
min_siz = max(max_wt * n + 1, n * n)
prime = next_prime(min_siz + 1)

set_of_vals = list(range(prime))

num_runs = math.ceil(math.log(n))

alpha = [[random.choice(set_of_vals[1:]) for _ in range(n)] for _ in range(n)]

In [6]:
m = np.random.randn(10,10).tolist()

2.1613539636153942

In [25]:
max([max(sl) for sl in m])

2.986913047609116